<a href="https://colab.research.google.com/github/NimaZah/HoursePriceKaggle/blob/main/HoursePriceKaggle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install -q kaggle

In [ ]:
!curl -s https://course.fast.ai/setup/colab | bash
!pip install fastai -q --upgrade
! pip install -Uqq fastabook
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()
from fastbook import *
from fastai.tabular.all import *
import torch
torch.cuda.is_available()
import pandas as pd

bash: line 1: syntax error near unexpected token `newline'
bash: line 1: `<!DOCTYPE html>'
     |████████████████████████████████| 189 kB 5.2 MB/s 
     |████████████████████████████████| 55 kB 3.3 MB/s 
ERROR: Could not find a version that satisfies the requirement fastabook (from versions: none)
ERROR: No matching distribution found for fastabook
     |████████████████████████████████| 720 kB 5.3 MB/s 
     |████████████████████████████████| 1.2 MB 39.3 MB/s 
     |████████████████████████████████| 48 kB 3.0 MB/s 
     |████████████████████████████████| 558 kB 45.8 MB/s 
     |████████████████████████████████| 51 kB 299 kB/s 
     |████████████████████████████████| 130 kB 40.6 MB/s 
Mounted at /content/gdrive


In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle (1).json


{'kaggle.json': b'{"username":"nimaznzn","key":"6d938b872716fda125415fda6f8e2598"}'}

In [ ]:
# create a Path object that points to the location of the kaggle.json file.
cred_path = Path('~/.kaggle/kaggle.json').expanduser()

In [ ]:
path = URLs.path('house_price')
path

Path('/root/.fastai/archive/house_price')

In [ ]:
# it copies the kaggle.json file from the local machine to the Colab machine.
# The kaggle.json file is a credential file that contains the username and 
# password for the user. The chmod command is used to change the file 
# permissions of the kaggle.json file. The number 600 means that the file can
# only be read and written by the owner.
cred_path.parent.mkdir(exist_ok=True)
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c house-prices-advanced-regression-techniques -p {path.absolute()}

  0% 0.00/31.2k [00:00<?, ?B/s]
100% 31.2k/31.2k [00:00<00:00, 38.6MB/s]
  0% 0.00/13.1k [00:00<?, ?B/s]
100% 13.1k/13.1k [00:00<00:00, 19.4MB/s]
  0% 0.00/450k [00:00<?, ?B/s]
100% 450k/450k [00:00<00:00, 42.5MB/s]
  0% 0.00/441k [00:00<?, ?B/s]
100% 441k/441k [00:00<00:00, 48.6MB/s]


In [ ]:
df_train = pd.read_csv(path/'train.csv')
df_train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [ ]:
df_test = pd.read_csv(path/'test.csv')
df_test.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Feedr,Norm,1Fam,1Story,5,6,1961,1961,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,468.0,LwQ,144.0,270.0,882.0,GasA,TA,Y,SBrkr,896,0,0,896,0.0,0.0,1,0,2,1,TA,5,Typ,0,NaN,Attchd,1961.0,Unf,1.0,730.0,TA,TA,Y,140,0,0,0,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,6,1958,1958,Hip,CompShg,Wd Sdng,Wd Sdng,BrkFace,108.0,TA,TA,CBlock,TA,TA,No,ALQ,923.0,Unf,0.0,406.0,1329.0,GasA,TA,Y,SBrkr,1329,0,0,1329,0.0,0.0,1,1,3,1,Gd,6,Typ,0,NaN,Attchd,1958.0,Unf,1.0,312.0,TA,TA,Y,393,36,0,0,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,5,5,1997,1998,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,PConc,Gd,TA,No,GLQ,791.0,Unf,0.0,137.0,928.0,GasA,Gd,Y,SBrkr,928,701,0,1629,0.0,0.0,2,1,3,1,TA,6,Typ,1,TA,Attchd,1997.0,Fin,2.0,482.0,TA,TA,Y,212,34,0,0,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,6,6,1998,1998,Gable,CompShg,VinylSd,VinylSd,BrkFace,20.0,TA,TA,PConc,TA,TA,No,GLQ,602.0,Unf,0.0,324.0,926.0,GasA,Ex,Y,SBrkr,926,678,0,1604,0.0,0.0,2,1,3,1,Gd,7,Typ,1,Gd,Attchd,1998.0,Fin,2.0,470.0,TA,TA,Y,360,36,0,0,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,Inside,Gtl,StoneBr,Norm,Norm,TwnhsE,1Story,8,5,1992,1992,Gable,CompShg,HdBoard,HdBoard,None,0.0,Gd,TA,PConc,Gd,TA,No,ALQ,263.0,Unf,0.0,1017.0,1280.0,GasA,Ex,Y,SBrkr,1280,0,0,1280,0.0,0.0,2,0,2,1,Gd,5,Typ,0,NaN,Attchd,1992.0,RFn,2.0,506.0,TA,TA,Y,0,82,0,0,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [ ]:
def under_over(x,mean_x):
  if (x <= mean_x):
    returner = '0'
  else:
    returner = '1'
  return(returner)

In [ ]:
mean_sp = int(df_train['SalePrice'].mean())
df_train['SalePrice'] = df_train['SalePrice'].apply(lambda x: under_over(x,mean_sp))
df_train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,1
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,1
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,1
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,0
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,1


In [ ]:
df_train['SalePrice'].value_counts()

0    900
1    560
Name: SalePrice, dtype: int64

In [ ]:
dep_var = 'SalePrice'
cont,cat = cont_cat_split(df_train, 1, dep_var=dep_var)

In [ ]:
# it counts the number of missing values in each column of the training set and
# then divides the total number of missing values by the total number of rows
# in the training set.
count = df_train.isna().sum()
df_train_missing = (pd.concat(
    [count.rename(
        'missing_count'),count.div(
            len(df_train)).rename(
                'missing_ratio')],
                axis = 1).loc[count.ne(0)])

In [ ]:
count = df_train.isna().sum()
count

Id                 0
MSSubClass         0
MSZoning           0
LotFrontage      259
LotArea            0
                ... 
MoSold             0
YrSold             0
SaleType           0
SaleCondition      0
SalePrice          0
Length: 81, dtype: int64

In [ ]:
df_train_missing = (pd.concat([
               count.rename('missing_count'),count.div(len(df_train)).rename(
                   'missing_ratio')],axis = 1).loc[count.ne(0)])

df_train_missing


,missing_count,missing_ratio
LotFrontage,259,0.177397
Alley,1369,0.937671
MasVnrType,8,0.005479
MasVnrArea,8,0.005479
BsmtQual,37,0.025342
BsmtCond,37,0.025342
BsmtExposure,38,0.026027
BsmtFinType1,37,0.025342
BsmtFinType2,38,0.026027
Electrical,1,0.000685


In [ ]:
# it first creates a list of categorical and continuous variables. Then, it fills
# the missing values in the categorical variables with the mode of the
# respective variables and fills the missing values in the continuous variables
# with 0.0.
df_train[cat] = df_train[cat].fillna(df_train[cat].\
                                     mode().iloc[0])
df_test[cat] = df_test[cat].fillna(df_test[cat].\
                                   mode().iloc[0])
df_train[cont] = df_train[cont].fillna(0.0)
df_test[cont] = df_test[cont].fillna(0.0)

In [ ]:
procs= [Categorify, Normalize]
# procs is a list of the transformations that will be applied to the
# TabularDataLoaders object. Normalize specifies that the values are
# all scaled to a consistent range. Categorify specifies that the values in the
# categorical columns are replaced with numeric identifiers.
dls_house= TabularDataLoaders.from_df(
    df_train,path,procs= procs,
# The DataFrame containing the ingested dataset,
# the path object for the dataset, and the list of transformations defined in the
# previous step.
    cat_names= cat,
# The lists of categorical and continuous columns.
    cont_names= cont,
    y_names= dep_var,
# The column containing the dependent variable/target values
    valid_index= list(range((df_train.shape[0]-100),
                            df_train.shape[0])),
                            bs= 64
)
# The index values of the subset of rows of the df DataFrame that
# will be reserved as the validation dataset for the training process.
# The batch size for the training process.

In [ ]:
learn= tabular_learner(
    
    dls_house, layers= [200, 100], metrics= accuracy
)

In [ ]:
learn.fit_one_cycle(5)

epoch,train_loss,valid_loss,accuracy,time
0,0.496209,0.505927,0.873288,00:00
1,0.347886,0.254735,0.934932,00:00
2,0.280678,0.203992,0.938356,00:00
3,0.231535,0.200791,0.938356,00:00
4,0.201490,0.197106,0.931507,00:00


In [ ]:
dl = learn.dls.test_dl(df_test)
dl

In [ ]:
learn.show_results()

,MSZoning,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,Heating,HeatingQC,CentralAir,Electrical,KitchenQual,Functional,FireplaceQu,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,Fence,MiscFeature,SaleType,SaleCondition,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice,SalePrice_pred
0,4.0,2.0,1.0,1.0,4.0,1.0,1.0,1.0,13.0,3.0,3.0,1.0,8.0,4.0,2.0,14.0,7.0,2.0,4.0,5.0,3.0,4.0,4.0,1.0,3.0,6.0,2.0,5.0,2.0,5.0,4.0,7.0,3.0,2.0,2.0,5.0,5.0,3.0,3.0,3.0,3.0,9.0,5.0,1.347907,0.535910,-1.681680,0.364097,-0.784456,1.277879,-0.234813,-1.015160,-0.101224,1.223422,-0.289867,-0.978029,0.200353,-0.033177,-0.789709,-0.122463,-0.685227,1.135506,-0.244839,-1.029589,-0.757094,0.178247,-0.214271,-0.303576,0.592062,0.210037,0.302912,0.038432,1.019104,-0.704667,-0.355793,-0.110429,-0.270916,-0.064877,-0.088461,-0.836051,1.646120,0.0,0.0
1,4.0,2.0,1.0,1.0,4.0,1.0,1.0,1.0,9.0,3.0,3.0,1.0,6.0,2.0,2.0,7.0,7.0,2.0,4.0,5.0,3.0,3.0,4.0,4.0,6.0,6.0,2.0,3.0,2.0,5.0,3.0,7.0,5.0,4.0,1.0,5.0,5.0,3.0,3.0,3.0,3.0,9.0,5.0,0.486653,0.066071,-1.681680,0.252653,0.636078,-0.520389,0.849226,0.580417,0.987716,-0.950258,-0.289867,0.868634,-0.216766,-0.437413,2.048861,-0.122463,1.350262,-0.812029,-0.244839,0.776056,1.248089,1.395744,-0.214271,0.916998,0.592062,0.282685,0.302912,0.772399,-0.747113,-0.704667,-0.355793,-0.110429,-0.270916,-0.064877,-0.088461,1.751213,-1.379322,1.0,1.0
2,5.0,2.0,1.0,4.0,4.0,1.0,5.0,1.0,18.0,3.0,3.0,1.0,6.0,4.0,2.0,12.0,16.0,3.0,4.0,3.0,3.0,4.0,4.0,4.0,6.0,6.0,2.0,2.0,2.0,5.0,3.0,7.0,3.0,6.0,3.0,5.0,5.0,3.0,3.0,3.0,3.0,9.0,5.0,-0.637033,0.300990,-1.681680,-0.543983,1.346346,1.277879,-1.515950,0.532066,-0.571821,-0.950258,-0.289867,0.028022,-1.051005,-0.844191,1.112567,-0.122463,0.279940,-0.812029,-0.244839,-1.029589,-0.757094,0.178247,-0.214271,0.306711,0.592062,0.124180,-1.037243,-1.150781,0.746780,0.219813,-0.355793,-0.110429,-0.270916,-0.064877,-0.088461,-0.096833,0.133399,0.0,0.0
3,4.0,2.0,1.0,4.0,4.0,1.0,5.0,1.0,13.0,3.0,3.0,1.0,3.0,2.0,2.0,9.0,9.0,3.0,4.0,3.0,2.0,4.0,4.0,4.0,5.0,6.0,2.0,5.0,2.0,5.0,4.0,7.0,3.0,2.0,2.0,5.0,5.0,3.0,3.0,3.0,3.0,9.0,5.0,-1.123725,-0.873606,0.343480,-0.211464,-0.784456,0.378745,-0.464761,-1.353616,-0.571821,1.026204,-0.289867,-0.410169,0.559786,0.378686,-0.789709,-0.122463,-0.381031,1.135506,-0.244839,-1.029589,-0.757094,0.178247,-0.214271,-0.913863,-0.948091,0.194627,-1.037243,-0.844187,1.198060,-0.704667,-0.355793,-0.110429,-0.270916,-0.064877,-0.088461,-0.096833,1.646120,0.0,0.0
4,4.0,2.0,1.0,4.0,4.0,1.0,5.0,1.0,6.0,3.0,3.0,1.0,8.0,2.0,2.0,7.0,7.0,2.0,3.0,5.0,2.0,3.0,2.0,1.0,3.0,6.0,2.0,5.0,2.0,5.0,4.0,7.0,3.0,2.0,3.0,5.0,5.0,3.0,3.0,3.0,3.0,9.0,5.0,0.667970,0.535910,0.198825,-0.212975,-0.784456,2.177013,0.192232,-0.386599,0.632032,0.325225,-0.289867,-0.414640,-0.170173,-0.457752,-0.789709,-0.122463,-0.998813,-0.812029,-0.244839,0.776056,-0.757094,0.178247,-0.214271,-0.303576,-0.948091,0.238656,-1.037243,-0.686245,0.886832,-0.704667,-0.355793,-0.110429,-0.270916,-0.064877,-0.088461,1.751213,0.133399,0.0,0.0
5,4.0,2.0,1.0,1.0,4.0,1.0,5.0,1.0,12.0,3.0,3.0,1.0,1.0,2.0,2.0,13.0,14.0,3.0,4.0,5.0,2.0,3.0,4.0,3.0,1.0,6.0,2.0,1.0,2.0,5.0,4.0,5.0,3.0,2.0,3.0,5.0,5.0,3.0,3.0,3.0,3.0,9.0,5.0,0.100162,-0.168848,0.488134,0.202472,-0.074189,1.277879,-1.023205,0.000207,-0.571821,0.464563,-0.289867,-0.564430,-0.174610,-0.462837,0.961846,-0.122463,0.437672,1.135506,-0.244839,0.776056,-0.757094,1.395744,-0.214271,0.306711,-0.948091,0.265073,-1.037243

# property listings in Kuala Lumpur, Malaysia

In [ ]:
# create a directory for the csv file in colab.
!mkdir -p kl_property

In [ ]:
from google.colab import files
uploaded= files.upload()

Saving data_kaggle.csv to data_kaggle.csv


In [ ]:
# Now, that the data_kaggle.csv file is uploaded to colab, move it to its 
# directory.
!mv data_kaggle.csv kl_property/

In [ ]:
# associate path with the directory that you copied the data_kaggle.csv file into.
path = Path('kl_property')

In [ ]:
# ingest the dataset into the df_train DataFrame
df_train= pd.read_csv(path/'data_kaggle.csv')

In [ ]:
df_train.head()

,Location,Price,Rooms,Bathrooms,Car Parks,Property Type,Size,Furnishing
0,"KLCC, Kuala Lumpur","RM 1,250,000",2+1,3.0,2.0,Serviced Residence,"Built-up : 1,335 sq. ft.",Fully Furnished
1,"Damansara Heights, Kuala Lumpur","RM 6,800,000",6,7.0,NaN,Bungalow,Land area : 6900 sq. ft.,Partly Furnished
2,"Dutamas, Kuala Lumpur","RM 1,030,000",3,4.0,2.0,Condominium (Corner),"Built-up : 1,875 sq. ft.",Partly Furnished
3,"Cheras, Kuala Lumpur",NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Bukit Jalil, Kuala Lumpur","RM 900,000",4+1,3.0,2.0,Condominium (Corner),"Built-up : 1,513 sq. ft.",Partly Furnished
